#### Parameter Grid Search

This code does broad grid search for each parameter (or combination if interdependant) seperately. Then, we repeat the grid searh, narrowing in on the minimum error and run time combinations.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import time

from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
from skopt.utils import use_named_args

from GA_params_class.GeneticAlgorithm import GeneticAlgorithm
from function_module import *

In [2]:
cnn_model_path = '../../Models/CNN_6_1_2.keras'
masked_sequence = 'AATACTAGAGGTCTTCCGACNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGTGTGGGCGGGAAGACAACTAGGGG'
target_expressions = [0, 0.5, 1]
precision = None
verbose = 0
lineages = 10
seed = 0

In [3]:
# Define the parameter ranges for skopt
# Does not include pop_size, islands, gene_flow_rate, num_competitors, boltzmann_temperature
param_ranges = [
    Integer(50, 500, name='generations'),
    Real(0.1, 1.0, name='base_mutation_rate'),
    Integer(1, 30, name='chromosomes'),
    Real(0.1, 1.0, name='elitist_rate'),
    Real(0.1, 1.0, name='surval_rate'),
    Integer(1, 10, name='num_parents'),
    Categorical(['tournament', 'tournament_pop', 'roulette', 
                 'linear_scaling', 'rank_based', 'sus', 'truncation'], name='selection')
]

In [4]:
# Objective function: combines error and runtime
@use_named_args(param_ranges)
def objective_function(**params):
    """
    Objective function for Bayesian Optimization.
    The function combines average_error and runtime.
    Lower weights favor faster runtime optimization.
    """
    average_error, average_runtime = bayesian_test(
        params,
        cnn_model_path,
        masked_sequence,
        target_expressions,
        precision,
        verbose,
        lineages,
        seed
    )

    # Combine error and runtime into a single objective
    weight_error = 0.7  # weight for the error
    weight_runtime = 0.3  # weight for the runtime
    
    combined_metric = weight_error * average_error + weight_runtime * average_runtime
    return combined_metric

In [5]:
# Perform Bayesian Optimization
result = gp_minimize(
    func=objective_function,
    dimensions=param_ranges,
    n_calls=30,  # Number of evaluations
    random_state=seed,
)

Testing params: {'generations': 317, 'base_mutation_rate': 0.8598391737229157, 'chromosomes': 26, 'elitist_rate': 0.8625265649057131, 'surval_rate': 0.6612073271073752, 'num_parents': 4, 'selection': 'roulette'}WARNING:tensorflow:From c:\Users\rsore\Documents\GitHub\promoter-sequence-transcription-rates\v2\Testing GA\GA_params\GA_params_class\Island.py:111: _EagerTensorBase.cpu (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.identity with explicit device placement instead.


c:\Users\rsore\anaconda3\envs\TX_prediction\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 34 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


 - Error: 0.21557579586903255, Run Time: 27.586507749557494
Testing params: {'generations': 76, 'base_mutation_rate': 0.3453906651221019, 'chromosomes': 15, 'elitist_rate': 0.8309518558979441, 'surval_rate': 0.5319794551375517, 'num_parents': 5, 'selection': 'sus'} - Error: 0.23064027825991315, Run Time: 6.660096287727356
Testing params: {'generations': 202, 'base_mutation_rate': 0.6833546848460775, 'chromosomes': 12, 'elitist_rate': 0.9614396430577419, 'surval_rate': 0.22631570237138066, 'num_parents': 9, 'selection': 'linear_scaling'} - Error: 0.21868026206890742, Run Time: 18.922218227386473
Testing params: {'generations': 410, 'base_mutation_rate': 0.5684297315960845, 'chromosomes': 21, 'elitist_rate': 0.7485693892533252, 'surval_rate': 0.6238178128675965, 'num_parents': 6, 'selection': 'sus'} - Error: 0.22822059045235318, Run Time: 37.39237017631531
Testing params: {'generations': 98, 'base_mutation_rate': 0.5262403774119918, 'chromosomes': 6, 'elitist_rate': 0.7632263594160624, '

c:\Users\rsore\anaconda3\envs\TX_prediction\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 34 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: Sample larger than population or is negative

In [ ]:
# Print the results
print("Optimal Parameters:")
print(f"Generations: {result.x[0]}")
print(f"Base Mutation Rate: {result.x[1]}")
print(f"Chromosomes: {result.x[2]}")
print(f"Elitist Rate: {result.x[3]}")
print(f"Survival Rate: {result.x[4]}")
print(f"Num Parents: {result.x[5]}")
print(f"Selection: {result.x[6]}")

print("\nBest Combined Metric:", result.fun)